In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_horizVert_wavelet_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,18632.05,0.10,0.15,-0.04,0.04,0.02,-0.00,-0.0
3,0.10,4981.68,-0.12,-0.02,0.01,-0.00,0.00,-0.0
4,0.15,-0.12,1578.04,-0.01,0.01,0.02,0.00,0.0
5,-0.04,-0.02,-0.01,448.60,-0.01,-0.01,-0.00,-0.0
6,0.04,0.01,0.01,-0.01,106.40,0.00,0.00,0.0
7,0.02,-0.00,0.02,-0.01,0.00,23.02,-0.00,0.0
8,-0.00,0.00,0.00,-0.00,0.00,-0.00,4.85,0.0
9,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.9


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00001,0.00003,-0.00001,0.00003,0.00004,-0.00002,-0.00002
3,0.00001,1.00000,-0.00004,-0.00001,0.00002,-0.00001,0.00002,-0.00006
4,0.00003,-0.00004,1.00000,-0.00001,0.00001,0.00008,0.00000,0.00001
5,-0.00001,-0.00001,-0.00001,1.00000,-0.00003,-0.00006,-0.00001,-0.00001
6,0.00003,0.00002,0.00001,-0.00003,1.00000,0.00002,0.00002,0.00001
7,0.00004,-0.00001,0.00008,-0.00006,0.00002,1.00000,-0.00002,0.00010
8,-0.00002,0.00002,0.00000,-0.00001,0.00002,-0.00002,1.00000,0.00002
9,-0.00002,-0.00006,0.00001,-0.00001,0.00001,0.00010,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.32027775967685335

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[4.34687832e+07 3.04438833e+06 2.96604279e+05 2.30025713e+04
 1.21503871e+03 5.08016283e+01 1.72156455e+00 6.08965548e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999082,-0.040970,-0.012010,-0.003393,-0.000802,-0.000173,-0.000037,-0.000007
1,0.040137,0.997339,-0.058898,-0.014922,-0.003504,-0.000759,-0.000159,-0.000030
2,0.014101,0.057279,0.996203,-0.062531,-0.013461,-0.002887,-0.000611,-0.000113
3,0.004797,0.018048,0.060509,0.995885,-0.063485,-0.012825,-0.002693,-0.000499
4,0.001411,0.005284,0.016737,0.061285,0.994950,-0.075983,-0.015060,-0.002780
5,0.000402,0.001508,0.004748,0.016781,0.072841,0.990950,-0.109753,-0.018982
6,0.000124,0.000464,0.001467,0.005167,0.021791,0.102975,0.977074,-0.184985
7,0.000047,0.000178,0.000559,0.001968,0.008291,0.038309,0.181789,0.982554


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0009178993739683561,
 0.002661293227289807,
 0.003797064765124203,
 0.004114641408020314,
 0.005049693951075462,
 0.009050137326121921,
 0.02292642414587709,
 0.01744602659101968]